# EDA & Pre-Cleaning of Employee Hiring History Dataset

## Pulling datasets from the database

In [137]:
# to read from My_Sql
import mysql.connector 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [138]:
# to import environment variables
from decouple import config, AutoConfig
config = AutoConfig(search_path='.env')

# To process special symbol in password
from urllib.parse import quote_plus

In [139]:
# Creating conncetion
# connection variables
# Mysql Credentials

host = config("HOST_MYSQL")
user = config("USER")
password = config("PASSWORD")
database = config("DATABASE")

In [140]:
conn = mysql.connector.connect(host=host,
                              user=user,
                              password=password,
                              database=database,
                              auth_plugin='mysql_native_password')

query_e = "SELECT * FROM raw_employee"
query_b = "SELECT * FROM raw_bu"

df_raw_emp = pd.read_sql(query_e, conn)
df_raw_bu = pd.read_sql(query_b, conn)

# commit the query
conn.commit()
conn.close()

In [141]:
# preserving the extracted dataset
df_e = df_raw_emp.copy()
df_b = df_raw_bu.copy()

## EDA - Exploratory Data Analysis

### Employee Dataset

#### Rows, Columns & Data

In [142]:
# dataset rows and columns size
df_e.shape

(1290259, 11)

In [143]:
df_e.head()

,date,EmpID,Gender,Age,EthnicGroup,FP,TermDate,BU,HireDate,PayTypeID,TermReason
0,2012-02-01,68346,D,21,1,P,None,9,2010-10-16,H,
1,2013-12-01,45108,D,21,1,P,None,9,2012-05-09,H,
2,2013-12-01,5582,D,21,1,P,None,9,2013-08-28,H,
3,2013-12-01,4708,D,21,1,P,None,9,2013-10-24,H,
4,2011-12-01,45486,D,21,1,P,None,9,2011-05-25,H,


In [144]:
df_e.tail()

,date,EmpID,Gender,Age,EthnicGroup,FP,TermDate,BU,HireDate,PayTypeID,TermReason
1290254,2013-07-01,113990,C,20,1,P,2013-07-25,99,2013-06-11,H,V
1290255,2013-08-01,117092,C,20,1,P,2013-08-30,99,2011-11-15,H,V
1290256,2013-10-01,120396,C,20,1,P,2013-10-12,99,2013-09-24,H,V
1290257,2013-12-01,13804,C,20,1,P,2013-12-17,99,2013-10-04,H,V
1290258,2014-05-01,24394,C,20,1,P,2014-05-23,99,2014-05-22,H,V


#### Data Type

In [145]:
df_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290259 entries, 0 to 1290258
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   date         1290259 non-null  object
 1   EmpID        1290259 non-null  int64 
 2   Gender       1290259 non-null  object
 3   Age          1290259 non-null  int64 
 4   EthnicGroup  1290259 non-null  int64 
 5   FP           1290259 non-null  object
 6   TermDate     29442 non-null    object
 7   BU           1290259 non-null  int64 
 8   HireDate     1290259 non-null  object
 9   PayTypeID    1290259 non-null  object
 10  TermReason   1290259 non-null  object
dtypes: int64(4), object(7)
memory usage: 108.3+ MB


#### Data Distribution

In [146]:
df_e.describe(include='all')

,date,EmpID,Gender,Age,EthnicGroup,FP,TermDate,BU,HireDate,PayTypeID,TermReason
count,1290259,1.290259e+06,1290259,1.290259e+06,1.290259e+06,1290259,29442,1.290259e+06,1290259,1290259,1290259
unique,48,NaN,2,NaN,NaN,2,734,NaN,7384,2,3
top,2014-10-01,NaN,D,NaN,NaN,P,2014-08-16,NaN,2012-08-24,H,
freq,33831,NaN,699620,NaN,NaN,659132,102,NaN,1604,1127691,1260817
mean,NaN,6.685939e+04,NaN,3.950015e+01,1.340873e+00,NaN,NaN,2.160108e+01,NaN,NaN,NaN
std,NaN,2.770236e+04,NaN,1.544600e+01,9.105428e-01,NaN,NaN,2.855896e+01,NaN,NaN,NaN
min,NaN,0.000000e+00,NaN,1.400000e+01,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN
25%,NaN,4.891400e+04,NaN,2.500000e+01,1.000000e+00,NaN,NaN,6.000000e+00,NaN,NaN,NaN
50%,NaN,6.853200e+04,NaN,3.900000e+01,1.000000e+00,NaN,NaN,1.300000e+01,NaN,NaN,NaN
75%,NaN,8.732800e+04,NaN,5.200000e+01,1.000000e+00,NaN,NaN,2.000000e+01,NaN,NaN,NaN


#### Null Values

In [147]:
# Total Null Values in each column
print(f'Total null values in each column of the dataframe = \n {df_e.isna().sum()}')

Total null values in each column of the dataframe = 
 date                 0
EmpID                0
Gender               0
Age                  0
EthnicGroup          0
FP                   0
TermDate       1260817
BU                   0
HireDate             0
PayTypeID            0
TermReason           0
dtype: int64


In [148]:
# Check the unique values of each row for each column
n = df_e.nunique(axis=0)
  
print("No.of.unique values in each column :\n",
      n)

No.of.unique values in each column :
 date              48
EmpID          61843
Gender             2
Age               83
EthnicGroup        7
FP                 2
TermDate         734
BU                30
HireDate        7384
PayTypeID          2
TermReason         3
dtype: int64


#### Unique / Distinct Values in Columns

In [149]:
# Checking if EmpID's are unique for each record

# Unique EmpID's
empid_unique = len(pd.unique(df_e['EmpID']))

print(f"Unique EMPIDs : {empid_unique}")
print(f"Are the EmpID's in each row unique: {len(set(df_e['EmpID'])) == df_e['EmpID'].count()}")

Unique EMPIDs : 61843
Are the EmpID's in each row unique: False


In [150]:
# Total termination dates available
len(df_e[df_e['TermDate'].notnull()])

29442

In [151]:
# Total termination dates with unique EmpID
len(pd.unique(df_e[df_e['TermDate'].notnull()]['EmpID']))

29442

In [152]:
# Looking at the first EmpID with a Termination Date
df_e[df_e['TermDate'].notnull()]['EmpID'].iloc[0]

120616

In [153]:
# Records of EmpID 120616
df_e.loc[df_e['EmpID'] == 120616]

,date,EmpID,Gender,Age,EthnicGroup,FP,TermDate,BU,HireDate,PayTypeID,TermReason
1206459,2012-02-01,120616,C,27,4,F,None,22,2007-03-01,S,
1206460,2013-06-01,120616,C,28,4,F,None,22,2007-03-01,S,
1206461,2011-02-01,120616,C,26,4,F,None,22,2007-03-01,S,
1206462,2011-01-01,120616,C,26,4,F,None,22,2007-03-01,S,
1206463,2013-05-01,120616,C,28,4,F,None,22,2007-03-01,S,
1206464,2012-08-01,120616,C,27,4,F,None,22,2007-03-01,S,
1206465,2012-09-01,120616,C,27,4,F,None,22,2007-03-01,S,
1206466,2011-04-01,120616,C,26,4,F,None,22,2007-03-01,S,
1206467,2011-03-01,120616,C,26,4,F,None,22,2007-03-01,S,
1206468,2012-07-01,120616,C,27,4,F,None,22,2007-03-01,S,


In [154]:
# Total Termination Dates
df_e['TermDate'].count() 

29442

#### Observations
* The dataset has `11 columns` and `1290259 rows`
* The count of  total null values in `TermReason` column says 0 but from the first 5 rows we can see that there are many null values
* On further analysis it was found that in the `TermReason` column the empty rows are not categorized as null
* Datatype of column `date` & `HireDate` is of type object which needs to be changed to type `Date`
* Column names are not appropriate Eg: `BU`
* Columns `Gender`, `EthnicGroup`, `FP`, `BU`, `TermReason` contain ID's and need to be renamed to reflect that
* `EmpID` cannot be primary key as this value pair is not unique
* Since the total number of employees is more than than the terminated ones this dataset containes employees who have not left the company as well
* Among the `TermDate` there is a unique termination date for each terminated employee

### BU (Business Unit) Dataset

In [155]:
df_b.shape

(30, 3)

In [156]:
df_b.head()

,BU,RegionSeq,VP
0,1,1-North,Sherley Rhymes
1,2,1-North,Dan Brown
2,3,1-North,Glennie Butters
3,4,1-North,Ruthann Lee
4,5,2-Midwest,Reena Hentz


In [157]:
df_b.tail()

,BU,RegionSeq,VP
25,95,1-North,Scott Salman
26,96,4-East,Patty Gallop
27,97,4-East,Jim Oyama
28,98,4-East,Anabell Hynes
29,99,4-East,Brett Jones


In [158]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   BU         30 non-null     int64 
 1   RegionSeq  30 non-null     object
 2   VP         30 non-null     object
dtypes: int64(1), object(2)
memory usage: 848.0+ bytes


In [159]:
df_b.describe(include='all')

,BU,RegionSeq,VP
count,30.000000,30,30
unique,NaN,7,30
top,NaN,1-North,Sherley Rhymes
freq,NaN,6,1
mean,29.300000,NaN,NaN
std,34.758427,NaN,NaN
min,1.000000,NaN,NaN
25%,8.250000,NaN,NaN
50%,15.500000,NaN,NaN
75%,22.750000,NaN,NaN


In [160]:
# Check the unique values of each row for each column
n = df_b.nunique(axis=0)
  
print("No.of.unique values in each column :\n",
      n)

No.of.unique values in each column :
 BU           30
RegionSeq     7
VP           30
dtype: int64


#### Observations:
* The BU Dataset name is not very discriptive - need to change it to BusinessUnit
* The `RegoinSeq` column can be split and added as a `Region` dimension table
* No null values in any column

## DATA CLEANING

### Employee Dataset

--- 

#### TermReason column
* Replace empty rows with null
* Convert id from `U` to `I` to represent Involuntary Termination

In [161]:
df_e['TermReason'].value_counts()

TermReason
     1260817
V      22048
U       7394
Name: count, dtype: int64

In [162]:
# 1260817 rows are empty
# Verifying if empty values are represented as null
df_e['TermReason'].isnull().sum()

0

In [163]:
# Checking if the empty values in the column TermReason is representend as empty string & counting them
df_e.loc[df_e['TermReason'] == '', ['TermReason']].count()

TermReason    1260817
dtype: int64

In [164]:
# Cleaning
# Replacing the empty rows with None
df_e['TermReason'].replace('', None, inplace=True)

In [165]:
# Verifying if empty string is replaced by none/null
df_e['TermReason'].isnull().sum()

1260817

In [166]:
# Replacing ID U with I
df_e['TermReason'].replace('U', 'I', inplace=True)

In [167]:
# Verifying Replacement
df_e['TermReason'].value_counts()

TermReason
V    22048
I     7394
Name: count, dtype: int64

---

#### Gender Column
* Changing Gender codes from
    * C to F (representing Female)
    * D to M (representing Male)

In [168]:
df_e['Gender'].value_counts()

Gender
D    699620
C    590639
Name: count, dtype: int64

In [169]:
df_e['Gender'].replace(to_replace={'C': 'F', 'D':'M'}, inplace=True)
df_e['Gender'].value_counts()

Gender
M    699620
F    590639
Name: count, dtype: int64

---

#### Date, HireDate and TermDate column 
* changing datatype

In [170]:
# Data type before cleaning
df_e.dtypes[['date', 'TermDate', 'HireDate']]

date        object
TermDate    object
HireDate    object
dtype: object

In [171]:
df_e['date'] = pd.to_datetime(df_e['date'])
df_e['TermDate'] = pd.to_datetime(df_e['TermDate'])
df_e['HireDate'] = pd.to_datetime(df_e['HireDate'])

In [172]:
# Data type after cleaning
df_e.dtypes[['date', 'TermDate', 'HireDate']]

date        datetime64[ns]
TermDate    datetime64[ns]
HireDate    datetime64[ns]
dtype: object

In [173]:
df_e[['date', 'TermDate', 'HireDate']][:5]

,date,TermDate,HireDate
0,2012-02-01,NaT,2010-10-16
1,2013-12-01,NaT,2012-05-09
2,2013-12-01,NaT,2013-08-28
3,2013-12-01,NaT,2013-10-24
4,2011-12-01,NaT,2011-05-25


---

#### Rename columns:
* BU to BusinessUnitID
* date to Date
* Gender to GenderID
* EthnicGroup to EthnicGroupID 
* FP to EmploymentTypeID
* TermReason to TermReasonID

In [174]:
# Column names before
df_e.columns

Index(['date', 'EmpID', 'Gender', 'Age', 'EthnicGroup', 'FP', 'TermDate', 'BU',
       'HireDate', 'PayTypeID', 'TermReason'],
      dtype='object')

In [175]:
# Column name dict
col_names_new = {'BU': 'BusinessUnitID', 
                       'date': 'Date', 
                       'Gender': 'GenderID', 
                       'EthnicGroup': 'EthnicGroupID', 
                       'FP': 'EmploymentTypeID',
                       'TermReason': 'TermReasonID'}
# Changing column names
df_e.rename(columns = col_names_new, inplace=True)

In [176]:
# Column names after
df_e.columns

Index(['Date', 'EmpID', 'GenderID', 'Age', 'EthnicGroupID', 'EmploymentTypeID',
       'TermDate', 'BusinessUnitID', 'HireDate', 'PayTypeID', 'TermReasonID'],
      dtype='object')

In [177]:
df_e.dtypes.to_dict()

{'Date': dtype('<M8[ns]'),
 'EmpID': dtype('int64'),
 'GenderID': dtype('O'),
 'Age': dtype('int64'),
 'EthnicGroupID': dtype('int64'),
 'EmploymentTypeID': dtype('O'),
 'TermDate': dtype('<M8[ns]'),
 'BusinessUnitID': dtype('int64'),
 'HireDate': dtype('<M8[ns]'),
 'PayTypeID': dtype('O'),
 'TermReasonID': dtype('O')}

### BU (Business Unit) Dataset

---

#### New Dataset Region

* Splitting RegionSeq column and adding to new 

In [178]:
df_r = pd.DataFrame()
df_r[['RegionID', 'Region']] = df_b['RegionSeq'].str.split(pat="-", n=1, expand=True)

In [179]:
df_r.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [180]:
df_r

,RegionID,Region
0,1,North
1,2,Midwest
2,6,South
3,5,Central
4,3,Northwest
5,7,West
6,4,East


---

#### RegionSeq column 

* Only extracting the RegionID from RegionSeq and adding to the dataset

In [181]:
df_b['RegionID'] = df_b['RegionSeq'].str.split(pat='-', n=1).str[0]
del df_b['RegionSeq']

---

#### BU column 

* Rename column to BusinessUnitID

In [182]:
df_b.rename(columns={'BU':'BusinessUnitID'}, inplace=True)

In [183]:
df_b

,BusinessUnitID,VP,RegionID
0,1,Sherley Rhymes,1
1,2,Dan Brown,1
2,3,Glennie Butters,1
3,4,Ruthann Lee,1
4,5,Reena Hentz,2
5,6,Aliza Fekete,2
6,7,Tom Benson,2
7,8,Brad Eagles,2
8,9,April Legolis,6
9,10,Maurine Krieger,6


## LOAD
* Write clean data to MySql Database

---

### Create table in the DB

#### Employee Dataset

In [184]:
# Creating a connection

conn = mysql.connector.connect(host=host,
                              user=user,
                              password=password,
                              database=database,
                              auth_plugin='mysql_native_password')
mycursor = conn.cursor()

In [185]:
# Create table to store clean data
query_drop_table = "drop table if exists Employee"
query_create_table = "CREATE TABLE Employee \
                                            (Date date, \
                                            EmpID integer, \
                                            GenderID char(1), \
                                            Age integer, \
                                            EthnicGroupID char(1), \
                                            EmploymentTypeID char(1), \
                                            TermDate date, \
                                            BusinessUnitID integer, \
                                            HireDate date, \
                                            PayTypeID char(1), \
                                            TermReasonID char(1))"

mycursor.execute(query_drop_table)
mycursor.execute(query_create_table)
#, Primary Key (EmpID, Date), foreign key (BusinessUnitID) references BusinessUnit (BusinessUnitID)

# commit the query
conn.commit()
# mycursor.close()
# conn.close()

#### BU (Business Unit) Dataset

In [186]:
query_drop_table = "drop table if exists BusinessUnit"
query_create_table = "CREATE TABLE BusinessUnit \
                                            (BusinessUnitID integer, \
                                            VP varchar(100), \
                                            RegionID integer)"

mycursor = conn.cursor()
mycursor.execute(query_drop_table)
mycursor.execute(query_create_table)
#,foreign key (RegionID) references Region (RegionID)
# commit the query
conn.commit()

#### Region Dataset

In [187]:
query_drop_table = "drop table if exists Region"
query_create_table = "CREATE TABLE Region \
                                            (RegionID integer, \
                                            Region varchar(100), \
                                            Primary Key (RegionID))"

mycursor = conn.cursor()
mycursor.execute(query_drop_table)
mycursor.execute(query_create_table)

# commit the query
conn.commit()

---

### Write Dataframe to DB
We use SQLAlchemy to write the dataframe to the table

#### Employee Dataset

In [188]:
# to write to My_Sql
import sqlalchemy as sq
from urllib.parse import quote_plus

connection_string = f"mysql+mysqlconnector://{user}:%s@{host}/{database}" % quote_plus(password)

engine = sq.create_engine(connection_string, echo=False)
conn_al = engine.connect()

In [189]:
schema_df_e = {
             'Date': sq.Date(),
             'EmpID': sq.types.INTEGER(),
             'GenderID': sq.types.CHAR(1),
             'Age': sq.types.INTEGER(),
             'EthnicGroupID': sq.types.CHAR(1),
             'EmploymentTypeID': sq.types.CHAR(1),
             'TermDate': sq.Date(),
             'BusinessUnitID':sq.types.INTEGER(),
             'HireDate': sq.Date(),
             'PayTypeID': sq.types.CHAR(1),
             'TermReasonID': sq.types.CHAR(1)
}

df_e.to_sql('employee', con=conn_al, if_exists='replace', index=False, dtype= schema_df_e, chunksize=2000)


1290259

#### Region Dataset

In [190]:
schema_df_r = {
             'RegionID': sq.types.INTEGER(),
              'Region': sq.types.VARCHAR(100)
}

df_r.to_sql('region', con=conn_al, if_exists='replace', index=False, dtype= schema_df_r, chunksize=2000)


7

#### BU (Business Unit) Dataset

In [191]:
schema_df_b = {
             'BusinessUnitID':sq.types.INTEGER(),
             'VP': sq.types.VARCHAR(100),
             'RegionID': sq.types.INTEGER()
}

df_b.to_sql('businessunit', con=conn_al, if_exists='replace', index=False, dtype= schema_df_b, chunksize=2000)
conn_al.close()

### Add Primary & Foreign Keys


In [192]:
# Region Table
query_alter_r = "ALTER TABLE region \
                                 ADD PRIMARY KEY (RegionID);"

In [193]:
# BusinessUnit Table
query_alter_bu = "ALTER TABLE businessunit \
                                 ADD PRIMARY KEY (BusinessUnitID), \
                                 ADD FOREIGN KEY (RegionID) \
                                 REFERENCES Region(RegionID);"

In [194]:
# Employee Table
query_alter_emp = "ALTER TABLE employee \
                                 ADD FOREIGN KEY (BusinessUnitID) \
                                 REFERENCES BusinessUnit(BusinessUnitID);"


In [195]:
mycursor = conn.cursor()
mycursor.execute(query_alter_r)
mycursor.execute(query_alter_bu)
mycursor.execute(query_alter_emp)

# commit the query
conn.commit()
conn.close()

## Testing

In [196]:
df_e.loc[df_e['EmpID']==15444]

,Date,EmpID,GenderID,Age,EthnicGroupID,EmploymentTypeID,TermDate,BusinessUnitID,HireDate,PayTypeID,TermReasonID
1246315,2013-12-01,15444,F,20,6,P,NaT,23,2013-12-18,H,None
1261493,2013-12-01,15444,F,20,6,P,2013-12-31,23,2013-12-18,H,V


In [66]:
df_e['TermDate'].isnull().sum()

1260817